# Preparing the environment


## Libraries

In [237]:
#%pip install --upgrade pip
#%pip install pandas
#%pip install scipy 
#%pip install scikit-learn 
#%pip install tqdm 
#%pip install plotly 
#%pip install matplotlib
#%pip install nbformat
#%pip install fastparquet
#%pip install pyarrow

In [238]:
# requirements
import pandas as pd
import numpy as np
from scipy.stats import chi2, poisson, chisquare
from scipy.stats import entropy  # for KL (use small-smoothing)
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, confusion_matrix
from tqdm import tqdm
import math, json
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.graph_objects as go

# Dataset 1 - Ravel Flow

These are network flows from period x - y, where DDoS attack were perceived.  



## Preparing the dataset
To make it useful for this project test, it was necessary to convert the columns name to the equivalent words used by the modules. The project assumes that the request information includes a HTTP method and an API endpoint, that is why the column ip.dst was converted to URI column.

In [ ]:
path_ravel = 'd:\BORDA_REDERIO_PTT\nfcapd.202511300000.csv'
df_ravel=pd.read_csv(path_ravel)